# Evaluating Numerical Expressions with Stacks

In this project, I will use the [stack](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) data structure to implement an algorithm that can evaluate numerical expressions.

For example, consider the expression $12\div(2+4)\cdot21$. To evaluate the result of this expression in Python, one simply needs to type it in the editor. However, behind the scenes, Python uses an algorithm to evaluate this expression.

The goal of this project is to use the stack data structure to implement a function named `evaluate()` that can evaluate expressions stored in a string, like so:
```python
expression = "12 / ( 2 + 4 ) * 21"
print(evaluate(expression))
```
```
42.0
```
## LinkedList & Stack Implementation

In [1]:
#importing LinkedList implementation
from linked_list import LinkedList

#Stack implementation
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
        
    def peek(self):
        return self.tail.data
    
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.head = self.tail = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Evaluating Expressions in Postfix Notation

For a computer, it's much easier to evaluate an expression written in postfix notation. For example, the infix expression `1 + 2` becomes `1 2 +` in postfix notation.

We can evaluate an expression in postfix notation using a stack. We read the expression from left to right and do the following:
1. If we find a number, push that number to the top of the stack.
2. If we find an operator, pop the top two elements of the stack, perform the operation, and then push back the result.

Let's implement a function `evaluate_postfix()` that evaluates an expression in postfix notation.

*Note: to simplify the function, we will assume the expression string has spaces between all elements in the expression.*

In [2]:
def tokenize(expression):
    return expression.split()

expression = "12 2 4 + / 21 *"
print(tokenize(expression))

['12', '2', '4', '+', '/', '21', '*']


The above function serves to tokenize a given expression, while the functions below will implement specific operators in said expression:

In [3]:
#processes '-'
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

#processes '+'
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = top + second_to_top
    stack.push(result)

#processes '*'    
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = top * second_to_top
    stack.push(result)

#processes '/'    
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)
    
#processes '**'    
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

Finally, to evaluate expressions in postfix notation, we need to do the following:

1. Tokenize the expression using the `tokenize()` function
2. Initialize an empty stack
3. For each token, do the following:
    1. If the token is an operator, call the corresponding function to process it (i.e. the `process_plus()` function for `+`).
    2. Otherwise, convert that number to `float` and push it to the top of the stack.
4. Return the value that is left in the stack.

In [4]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == '+':
            process_plus(stack)
        elif token == '-':
            process_minus(stack)
        elif token == '*':
            process_times(stack)
        elif token == '/':
            process_divide(stack)
        elif token == '**':
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()

Test expressions:

In [5]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Converting Expressions from Infix to Postfix

To convert an expression from infix to postfix, I'm going to implement the [Shunting-yard algorithm](https://en.wikipedia.org/wiki/Shunting_yard_algorithm#:~:text=In%20computer%20science%2C%20the%20shunting,abstract%20syntax%20tree%20(AST).) in a function called `infix_to_postfix()`, which is also implemented with a stack.

To start, I need to create a dictionary defining the precedence of each operator according to order of operations:

In [6]:
precedence = {
    '+': 1,
    '-': 1,
    '*': 2,
    '/': 2,
    '**': 3
}

print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

True
False
True


The processing for each token in `infix_to_postfix()` should be as follows:
* Opening parenthesis, `(`:
    1. Push the token into the stack for later use when closing parenthesis is found.
* Closing parenthesis, `)`:
    1. While the top of the stack isn't an opening parenthesis, pop the top element and append it to the `postfix` token list.
    2. Pop the opening parenthesis out of the stack at the end.
* Operator:
    1. While the top of the stack is also an operator w/ a precedence greater than or equal to this operator, pop the top element, and append it to the `postfix` token list.
    2. Push the current operator to the top of the stack.
* Operand:
    1. Append the number to the `postfix` token list.

In [7]:
#processing '('
def process_opening_parenthesis(stack):
    stack.push('(')
    
#processing ')'
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != '(':
        postfix.append(stack.pop())
    stack.pop()
    
#processing '+', '-', '*', '/', '**'
def process_operator(stack, postfix, operator):
    while (len(stack) > 0 
           and stack.peek() in precedence 
           and precedence[stack.peek()] >= precedence[operator]
          ):
        postfix.append(stack.pop())
    stack.push(operator)
    
#processing numbers
def process_number(postfix, number):
    postfix.append(number)

Finally, the `infix_to_postfix()` function can be implemented to convert an expression from infix notation to postfix notation.

This function will work as follows:
1. Tokenize the expression using the `tokenize()` function.
2. Initialize an empty stack.
3. Initialize an empty postfix token list.
4. Iterate over all tokens, and call the appropriate processing functions (above) as needed.
5. Pop the remaining stack element into the postfix token list.
6. Convert the postfix token list into a string.

In [8]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == '(':
            process_opening_parenthesis(stack)
        elif token == ')':
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return ' '.join(postfix)

## Conclusion: Evaluating Infix Expressions

With a function that converts expressions to postfix notation and another to evaluate them, we can combine the two to evaluate an infix expression:

In [9]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

Test expressions:

In [10]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
